# Topic modeling of Tweets
I have a pretty good model here - it runs wonderfully currently. I can train it on what it's training on and test it with a sample of my tweets.

In [1]:
#text_file = "Corona_NLP_train.csv" # EDIT THIS WITH THE FILE YOU UPLOAD

import pandas as pd
import re
import spacy
import string
from string import digits
import nltk

#d = pd.read_csv("without_group1_ready.csv", lineterminator='\n')#, sep= ',', encoding = 'ansi', error_bad_lines = False)
#d = d.dropna()

file = open("../hope-b117/stop_words.txt","r+")
stop_words = file.read().split()

sp = spacy.load('da_core_news_lg')

In [2]:
import ndjson
import os
import re
import pandas as pd
# path to files to process
dir_path = "/data/001_twitter_hope/preprocessed/da"
# list all ndjson files, sort the list so latest come first
filenames = [f for f in os.listdir(dir_path) if f.endswith(".ndjson") and f.startswith("td_202102")]

df = pd.DataFrame()
for f in filenames:
    path = os.path.join(dir_path, f)
    with open(path, "r") as fname:
        data = ndjson.load(fname)
    df0 = pd.DataFrame(data)
    df = pd.concat([df, df0])

In [3]:
df = df[0:100]

In [4]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def clean_tweets(row):
    tweet = row["text"].lower()
    
    #clean_tweet = re.sub(r'^RT', '', tweet) #RT
    clean_tweet = re.sub(r'@(\S*)\w', '', tweet) #mentions
    #clean_tweet = re.sub(r'#\S*\w', '', clean_tweet) #hashtags
    # Remove URLs
    url_pattern = re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')
    clean_tweet = re.sub(url_pattern, '', clean_tweet)
    
    clean_tweet = remove_emoji(clean_tweet)
    
    #clean_tweet = clean_tweet.translate(str.maketrans('', '', string.punctuation))
    #clean_tweet = clean_tweet.replace('”', '')
    #clean_tweet = clean_tweet.replace('“', '')
    
    return clean_tweet
    
    # Remove stopwords
    #words = clean_tweet.split()
    #res = [x for x in words if x not in stop_words]
    
    #sentence = sp(' '.join(res))
    #lemmas = []
    #for word in sentence:
    #    lemmas.append(word.lemma_)
    
    #if len(lemmas) > 3:
    #    return ' '.join(lemmas)
    #else:
    #    return 1
    
def retrieve_retweets(row):
    if re.match("^RT", row):
        RT = True
    else:
        RT = False
    return RT

def remove_retweets(data):
    patternDel = "^RT"
    data["text"] = data["text"].astype(str)
    filtering = data['text'].str.contains(patternDel)
    removed_RT = data[~filtering].reset_index(drop=True)
    
    return removed_RT

def remove_mention_tweets(data):
    patternDel = "^@"
    data["text"] = data["text"].astype(str)
    filtering = data['text'].str.contains(patternDel)
    removed_RT = data[~filtering].reset_index(drop=True)
    
    return removed_RT

In [6]:
df = remove_retweets(df)
print(len(df))
df = remove_mention_tweets(df)
print(len(df))
df["clean_text"] = df.apply(lambda row: clean_tweets(row), axis = 1)
df = df[df["clean_text"] != 1].reset_index(drop=True)

documents = df["clean_text"].to_list()

27
27


In [7]:
df

,id,created_at,text,retweet_count,favorite_count,lang,from_user_id,clean_text
0,1359653488930787329,2021-02-11 00:00:27,Dejligt mange talenter på bænken. Jeg håber at...,0,0,da,1329784996811460609,dejligt mange talenter på bænken. jeg håber at...
1,1359653862420975622,2021-02-11 00:01:56,Mor får bøde for at risikere 3 børns liv (og d...,0,0,da,1655769439,mor får bøde for at risikere 3 børns liv (og d...
2,1359654029715005440,2021-02-11 00:02:36,Mig: gør ingenting hele dagen... Mine følgere...,0,0,da,989120648344555520,mig: gør ingenting hele dagen... mine følgere:
3,1359654112661557248,2021-02-11 00:02:56,En dag træner jeg en falk til at stjæle cykell...,0,0,da,1294417944240422918,en dag træner jeg en falk til at stjæle cykell...
4,1359654133100331008,2021-02-11 00:03:01,fuck jer @FortniteGame så tager i bare mine ra...,0,0,da,1341744032490418178,fuck jer så tager i bare mine rare skins tilb...
5,1359654172472340480,2021-02-11 00:03:10,Hvis et Zoom-møde var fysisk https://t.co/eVj1...,0,0,da,4210444288,hvis et zoom-møde var fysisk
6,1359654224666243075,2021-02-11 00:03:22,Aktivisten Loujain al Hathloul blev anholdt i ...,0,0,da,14930764,aktivisten loujain al hathloul blev anholdt i ...
7,1359654229879717892,2021-02-11 00:03:24,"okay klokken er 1,.. karla har måske en pointe 😣",0,0,da,1279529293933154307,"okay klokken er 1,.. karla har måske en pointe"
8,1359654499170844673,2021-02-11 00:04:28,Folk gemmer deres morgenjern men har offentlig...,0,0,da,1294721576458387458,folk gemmer deres morgenjern men har offentlig...
9,1359654614388400128,2021-02-11 00:04:55,En flink person på twitter som vil gøre mig gl...,0,0,da,1133722079348568064,en flink person på twitter som vil gøre mig gl...


In [8]:
df["date"] = pd.to_datetime(df["created_at"], utc=True).dt.strftime('%Y-%m-%d')

import re

def extract_hashtags(row):
    unique_hashtag_list = list(re.findall(r'#\S*\w', row["text"]))
    return unique_hashtag_list

def hashtag_per_row(data):
    # Create hashtags column with the actual unique hashtags
    data["hashtags"] = data.apply(lambda row: extract_hashtags(row), axis = 1)

    # Let's take a subset of necessary columns, add id
    df = data[["date", "hashtags"]].reset_index().rename(columns={"index": "id"})

    # Select only the ones where we have more than 1 hashtag per tweet
    df = df[df["hashtags"].map(len) > 1].reset_index(drop=True)

    # Hashtag per row
    # convert list of pd.Series then stack it
    df = (df
     .set_index(['date','id'])['hashtags']
     .apply(pd.Series)
     .stack()
     .reset_index()
     .drop('level_2', axis=1)
     .rename(columns={0:'hashtag'}))
    #lowercase!
    df["hashtag"] = df["hashtag"].str.lower()
    df["hashtag"] = df["hashtag"].str.replace("'.", "")
    df["hashtag"] = df["hashtag"].str.replace("’.", "")

    return df

hashtags = hashtag_per_row(df)

In [9]:
# Aggregate a frequency DF
def get_hashtag_frequencies(df):
    # Add freq of hashtags by themselves in the dataset
    tweet_freq = pd.DataFrame({'nr_of_hashtags' : df.groupby(['hashtag']).size()}).reset_index()
    return tweet_freq
freq_hashtags = get_hashtag_frequencies(hashtags)
freq_hashtags = freq_hashtags.sort_values(by=['nr_of_hashtags'], ascending=False)

In [10]:
hashtags_to_rmv = freq_hashtags.iloc[0:30]["hashtag"].to_list()

In [11]:
hashtags_to_rmv

['#copaargentina', '#riverplatedk']

In [13]:
#df = pd.read_csv("clean_TobiasPetersen_topic_model_testset.csv").rename(columns={"0": "created_at",
#                                                                                "1": "id",
#                                                                                "2": "text",
#                                                                                "3": "from_user_id"})
def clean_tweets(row):
    #tweet = row["clean_text"] thesis one
    tweet = row["clean_text"]
    # Remove stopwords
    words = tweet.split()
    clean_tweet = [x for x in words if x not in hashtags_to_rmv]
    #clean_tweet = re.sub(r'^rt', '', tweet) #RT
    #clean_tweet = re.sub(r'[0-9]+', '', clean_tweet) # numbers
    
    clean_tweet = ' '.join(clean_tweet)
    clean_tweet = re.sub(r'#\S*\w', '', clean_tweet) #hashtags
    clean_tweet = clean_tweet.translate(str.maketrans('', '', string.punctuation))
    clean_tweet = clean_tweet.replace('”', '')
    clean_tweet = clean_tweet.replace('“', '')
    
    # Remove stopwords
    clean_tweet = clean_tweet.split()
    res = [x for x in clean_tweet if x not in stop_words]
    #print(ff)
    if len(res) > 3:
        sentence = sp(' '.join(res))
        lemmas = []
        for word in sentence:
            lemmas.append(word.lemma_)
    
        if len(lemmas) > 3:
            return ' '.join(lemmas)
        else:
            return 1
    
    #if len(clean_tweet) > 3:
    #    return " ".join(clean_tweet)
    #else:
    #    return 1
    
df["cleaner_text"] = df.apply(lambda row: clean_tweets(row), axis = 1)
    
print(len(df))
print(len(df[df["cleaner_text"] != 1]))

df = df[df["cleaner_text"] != 1]
italian_documents = df["cleaner_text"].to_list()

27
27


In [14]:
df = df.dropna()

In [15]:
italian_documents = df["cleaner_text"].to_list()

In [16]:
def remove_4(row):
    lemmas = row["cleaner_text"].split()
    if len(lemmas) > 4:
        return ' '.join(lemmas)
    else:
        return 1

df["above_4"] = df.apply(lambda row: remove_4(row), axis = 1)
    
print(len(df))
print(len(df[df["above_4"] != 1]))

df = df[df["above_4"] != 1]
italian_documents = df["above_4"].to_list()

24
21


In [17]:
#df["above_4"]

In [18]:
len(df)

21

In [19]:
df.iloc[2]["cleaner_text"]

'dag træne falk stjæle cykellygte fandeme hævn'

In [20]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [21]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/commando/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
#documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(italian_documents, stopwords_language='danish') #try training on the same dataset as I later do it on
#sp = WhiteSpacePreprocessing(documents, stopwords_language='danish')

preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

In [23]:
preprocessed_documents[:2]

['dejlig talent aktion',
 'mor risikere barn liv desuden bryde lov jarlov angribe fri samfund cant make this shit up']

In [24]:
tp = TopicModelDataPreparation("distiluse-base-multilingual-cased")

training_dataset = tp.create_training_set(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
tp.vocab[:10]

['aarhus',
 'agenda',
 'aktion',
 'aktiv',
 'aktivist',
 'al',
 'alder',
 'altid',
 'angribe',
 'anholde']

In [26]:
ctm = ZeroShotTM(input_size=len(tp.vocab), bert_input_size=512, n_components=50, num_epochs=15)
ctm.fit(training_dataset) # run the model

Epoch: [15/15]	 Seen Samples: [315/315]	Train Loss: 88.65764508928571	Time: 0:00:01.873435: : 15it [00:28,  1.87s/it]


In [27]:
ctm.get_topic_lists(5)

[['menneske', 'kue', 'finte', 'vant', 'satse'],
 ['miste', 'satse', 'pinkie', 'angribe', 'chance'],
 ['fri', 'ha', 'tromle', 'up', 'aarhus'],
 ['rettighed', 'forstaa', 'samfund', 'service', 'kontor'],
 ['nogeten', 'bryde', 'tid', 'officiel', 'godnat'],
 ['fuck', 'talent', 'putins', 'film', 'tesla'],
 ['cykellygte', 'hendesfarogstuefarog', 'hurtig', 'translate', 'scarlet'],
 ['service', 'ramme', 'splitte', 'miste', 'misse'],
 ['anholde', 'hathloul', 'arbejde', 'snart', 'faste'],
 ['think', 'liv', 'google', 'faste', 'glad'],
 ['focus', 'skrive', 'bryde', 'bitter', 'hathloul'],
 ['focus', 'nogeten', 'chance', 'alder', 'officiel'],
 ['loujain', 'diverse', 'fandeme', 'kue', 'al'],
 ['tweete', 'endelig', 'rette', 'engang', 'medlem'],
 ['samfund', 'unge', 'halvleg', 'falk', 'social'],
 ['bet', 'offentlig', 'social', 'its', 'talent'],
 ['russisk', 'kue', 'hav', 'minut', 'kort'],
 ['shit', 'fysisk', 'snuble', 'hurtig', 'shoppe'],
 ['skrive', 'liv', 'pinkie', 'hathloul', 'bilist'],
 ['twitter', 

In [28]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

/home/commando/maris/lib/python3.6/site-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:09,  1.92s/it]


In [29]:
preprocessed_documents[0] # see the text of our preprocessed document

'dejlig talent aktion'

In [30]:
import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [31]:
ctm.get_topic_lists(10)[topic_number] #and the topic should be about natural location related things

['burn',
 'its',
 'dum',
 'mor',
 'god',
 'rekord',
 'big',
 'bitcoinprisen',
 'splitte',
 'forbud']

## Testing

In [205]:
# path to files to process
dir_path = "/data/001_twitter_hope/preprocessed/da"
# list all ndjson files, sort the list so latest come first
filenames = [f for f in os.listdir(dir_path) if f.endswith(".ndjson") and f.startswith("td_20210102")]

df = pd.DataFrame()
for f in filenames:
    path = os.path.join(dir_path, f)
    with open(path, "r") as fname:
        data = ndjson.load(fname)
    df0 = pd.DataFrame(data)
    df = pd.concat([df, df0])

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def clean_tweets(row):
    tweet = row["text"].lower()
    
    #clean_tweet = re.sub(r'^RT', '', tweet) #RT
    clean_tweet = re.sub(r'@(\S*)\w', '', tweet) #mentions
    #clean_tweet = re.sub(r'#\S*\w', '', clean_tweet) #hashtags
    # Remove URLs
    url_pattern = re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')
    clean_tweet = re.sub(url_pattern, '', clean_tweet)
    
    clean_tweet = remove_emoji(clean_tweet)
    
    clean_tweet = clean_tweet.translate(str.maketrans('', '', string.punctuation))
    clean_tweet = clean_tweet.replace('”', '')
    clean_tweet = clean_tweet.replace('“', '')
    
    return clean_tweet
    
    # Remove stopwords
    #words = clean_tweet.split()
    #res = [x for x in words if x not in stop_words]
    
    #sentence = sp(' '.join(res))
    #lemmas = []
    #for word in sentence:
    #    lemmas.append(word.lemma_)
    
    #if len(lemmas) > 3:
    #    return ' '.join(lemmas)
    #else:
    #    return 1
    
def retrieve_retweets(row):
    if re.match("^RT", row):
        RT = True
    else:
        RT = False
    return RT

def remove_retweets(data):
    patternDel = "^RT"
    data["text"] = data["text"].astype(str)
    filtering = data['text'].str.contains(patternDel)
    removed_RT = data[~filtering].reset_index(drop=True)
    
    return removed_RT
    
df = remove_retweets(df)
df["clean_text"] = df.apply(lambda row: clean_tweets(row), axis = 1)
df = df[df["clean_text"] != 1].reset_index(drop=True)

#documents = d["clean_text"].to_list()
#df["cleaner_text"] = df.apply(lambda row: clean_tweets(row), axis = 1)
    
#print(len(df))
#print(len(df[df["cleaner_text"] != 1]))

#df = df[df["cleaner_text"] != 1]
#test_documents = df["cleaner_text"].to_list()

In [206]:
df

,id,created_at,text,retweet_count,favorite_count,lang,from_user_id,clean_text
0,1345157866446069762,2021-01-02 00:00:01,Vi tager en lørdag #lørdag,0,0,da,267854711,vi tager en lørdag lørdag
1,1345157876961202176,2021-01-02 00:00:04,"Hvad jeg ser, når jeg får reklamer for Mr. Gre...",0,0,da,4171496961,hvad jeg ser når jeg får reklamer for mr green...
2,1345157920682598401,2021-01-02 00:00:14,@ASElliotKing Phew det er galt nok uden at Ven...,0,0,da,891428135618707456,phew det er galt nok uden at venstre går am...
3,1345157922079256578,2021-01-02 00:00:15,@data02 @Fiscker89 @mikaelmilhoj Ikke ved den ...,0,0,da,478372813,ikke ved den første forsendelse i hvert fal...
4,1345157926831448064,2021-01-02 00:00:16,@nboegekaer Boss plan #nemt Hilsen udkant med ...,0,0,da,1297922966488047617,boss plan nemt hilsen udkant med lejehus på o...
...,...,...,...,...,...,...,...,...
24767,1345520154835378176,2021-01-02 23:59:38,"@Krisandthebike @YaelBassan Ja, så fastholder ...",0,0,da,82435798,ja så fastholder man stednavn og det lokale ...
24768,1345520161151983616,2021-01-02 23:59:39,"@NiclasBekker @YaelBassan Gå en tur - nej, tag...",0,0,da,2505741481,gå en tur nej tag din cykel og kør en tur t...
24769,1345520165509709824,2021-01-02 23:59:40,"@magneme @AudunHBerge Eller, forresten. Jeg va...",0,0,da,30270192,eller forresten jeg var på bob også
24770,1345520214075703298,2021-01-02 23:59:52,"@DavidTrads Hvad skal vi med politikere, når d...",0,0,da,47115984,hvad skal vi med politikere når de konstant u...


In [207]:
def clean_tweets(row):
    #tweet = row["clean_text"] thesis one
    tweet = row["clean_text"]
    # Remove stopwords
    words = tweet.split()
    clean_tweet = [x for x in words if x not in hashtags_to_rmv]
    #clean_tweet = re.sub(r'^rt', '', tweet) #RT
    #clean_tweet = re.sub(r'[0-9]+', '', clean_tweet) # numbers
    
    # Remove stopwords
    #words = clean_tweet.split()
    res = [x for x in clean_tweet if x not in stop_words]
    
    sentence = sp(' '.join(res))
    lemmas = []
    for word in sentence:
        lemmas.append(word.lemma_)
    
    if len(lemmas) > 4:
        return ' '.join(lemmas)
    else:
        return 1
    
    #if len(clean_tweet) > 3:
    #    return " ".join(clean_tweet)
    #else:
    #    return 1
    
df["cleaner_text"] = df.apply(lambda row: clean_tweets(row), axis = 1)
    
print(len(df))
print(len(df[df["cleaner_text"] != 1]))

df = df[df["cleaner_text"] != 1]
test_documents = df["cleaner_text"].to_list()

24772
17570


In [208]:
test_documents = df["cleaner_text"].to_list()

In [210]:
#df["cleaner_text"][1000]

In [211]:
testing_dataset = tp.create_test_set(test_documents) # create dataset for the testset
# n_sample how many times to sample the distribution (see the documentation)
italian_topics_predictions = ctm.get_thetas(testing_dataset, n_samples=5) # get all the topic predictions

Batches:   0%|          | 0/88 [00:00<?, ?it/s]

/home/commando/maris/lib/python3.6/site-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:26,  5.29s/it]


In [212]:
topic_number = np.argmax(italian_topics_predictions[0]) # get the topic id of the first document
ctm.get_topic_lists(10)[topic_number]

['kr',
 'million',
 'pris',
 'mio',
 'koste',
 'krone',
 'mia',
 'milliard',
 'billig',
 'aktie']

In [213]:
topics = []
for tweet in range(0, len(test_documents)):
    topic_number = np.argmax(italian_topics_predictions[tweet]) # get the topic id of the first document
    topics.append(ctm.get_topic_lists(10)[topic_number])

In [214]:
df["topics"] = topics

/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [215]:
df["topics_str"] = df["topics"].astype(str)
len(df.drop_duplicates("topics_str"))

/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


50

In [216]:
dff = df[["created_at", "id", "text", "from_user_id", "topics_str"]]
topics_df = dff.drop_duplicates("topics_str").reset_index()
topics_df = topics_df.rename(columns={"index":"topic_nr"})
merged = pd.merge(dff, topics_df, how="left", on=["topics_str"])
merged = merged.drop(["created_at_y", "id_y", "text_y", "from_user_id_y"], axis = 1).rename(columns={"created_at_x":"created_at", "id_x":"id","text_x":"text","from_user_id_x":"from_user_id"})
merged = merged.drop_duplicates().reset_index(drop=True)

In [217]:
merged

,created_at,id,text,from_user_id,topics_str,topic_nr
0,2021-01-02 00:00:04,1345157876961202176,"Hvad jeg ser, når jeg får reklamer for Mr. Gre...",4171496961,"['kr', 'million', 'pris', 'mio', 'koste', 'kro...",1
1,2021-01-02 00:00:14,1345157920682598401,@ASElliotKing Phew det er galt nok uden at Ven...,891428135618707456,"['kvinde', 'barn', 'dansk', 'hjem', 'mand', 'd...",2
2,2021-01-02 00:00:15,1345157922079256578,@data02 @Fiscker89 @mikaelmilhoj Ikke ved den ...,478372813,"['menneske', 'folk', 'liv', 'leve', 'corona', ...",3
3,2021-01-02 00:00:16,1345157926831448064,@nboegekaer Boss plan #nemt Hilsen udkant med ...,1297922966488047617,"['kr', 'million', 'pris', 'mio', 'koste', 'kro...",1
4,2021-01-02 00:00:24,1345157960406863875,Årets første byggeansøgning er sendt afsted. #...,202824011,"['jobdk', 'palle', 'kaffe', 'blad', 'kat', 'dr...",5
...,...,...,...,...,...,...
17565,2021-01-02 23:59:11,1345520041991823360,@stinelinnemann @RunestenConsult @MLundbyHanse...,1210998496968331264,"['dansk', 'danmark', 'dansker', 'land', 'stats...",84
17566,2021-01-02 23:59:38,1345520154835378176,"@Krisandthebike @YaelBassan Ja, så fastholder ...",82435798,"['kold', 'sne', 'varme', 'sidde', 'bil', 'vand...",14
17567,2021-01-02 23:59:39,1345520161151983616,"@NiclasBekker @YaelBassan Gå en tur - nej, tag...",2505741481,"['kold', 'sne', 'varme', 'sidde', 'bil', 'vand...",14
17568,2021-01-02 23:59:52,1345520214075703298,"@DavidTrads Hvad skal vi med politikere, når d...",47115984,"['lov', 'stemme', 'folketing', 'politiker', 'r...",19


In [240]:
merged.drop_duplicates("topics_str")#["topics_str"]

,created_at,id,text,from_user_id,topics_str,topic_nr
0,2021-01-02 00:00:04,1345157876961202176,"Hvad jeg ser, når jeg får reklamer for Mr. Gre...",4171496961,"['kr', 'million', 'pris', 'mio', 'koste', 'kro...",1
1,2021-01-02 00:00:14,1345157920682598401,@ASElliotKing Phew det er galt nok uden at Ven...,891428135618707456,"['kvinde', 'barn', 'dansk', 'hjem', 'mand', 'd...",2
2,2021-01-02 00:00:15,1345157922079256578,@data02 @Fiscker89 @mikaelmilhoj Ikke ved den ...,478372813,"['menneske', 'folk', 'liv', 'leve', 'corona', ...",3
4,2021-01-02 00:00:24,1345157960406863875,Årets første byggeansøgning er sendt afsted. #...,202824011,"['jobdk', 'palle', 'kaffe', 'blad', 'kat', 'dr...",5
6,2021-01-02 00:00:42,1345158038743822341,@t_hebsgaard Fint. Så skal vi ikke tale om fly...,484074783,"['gode', 'god', 'tak', 'rigtig', 'virkelig', '...",7
7,2021-01-02 00:00:49,1345158065763606528,@flyvepige da også den ENESTE legit grund.. Er...,1215826445328621568,"['land', 'folk', 'verden', 'lukke', 'liv', 'me...",8
8,2021-01-02 00:00:50,1345158071140683776,@KVarming Ja og det gik ekstra stærkt efter de...,1228743965089943554,"['smitte', 'vaccinere', 'vaccine', 'mutation',...",9
9,2021-01-02 00:01:05,1345158134000742402,@pelledragsted Og midt i valgkampen enerådigt ...,2431637931,"['skifte', 'skift', 'batteri', 'christian', 'k...",11
10,2021-01-02 00:01:06,1345158136785760259,"En hovedpine mere for Ellemann, der står med e...",3091463621,"['dkpol', 'dkmedier', 'corona', 'unge', 'barn'...",12
11,2021-01-02 00:01:14,1345158172953214976,@tv2newsdk Straffen burde være hjemsendelse.. ...,1296196124735307776,"['kold', 'sne', 'varme', 'sidde', 'bil', 'vand...",14


In [241]:
number = 14
merged[merged["topic_nr"] == number]["text"].to_list()

['@tv2newsdk Straffen burde være hjemsendelse.. så stoppede det helt, når de var sendt hjem',
 '@Kildegaardens Hvis LLR forlader Folketinget, hvem får så hans plads, nu hvor han er løsgænger?',
 'Så det var en skidt start på 2021. Kørt på sygehuset da jeg ikke kunne trække vejret pga smerter i ryggen på hver indånding. Er hjemme igen med smertestillende og muskel afslappende 👌 Men shit en nederen oplevelse at ikke kunne trække vejret.  It can only go up from here. https://t.co/StdjecyOm4',
 '@kikros @jakob_dahl @RasmusMalver Du har ret! Det er der elastikken er. Og så kommer vi jo nok heller ikke udenom, at de græske cigaretter, som hun røg/ryger(?), nok også har en effekt på hendes modstandskraft over for en omgang Covid',
 "@ChristianWolny @dr2tv fornøjelse, nok 50x jeg så den, så mange gode citater  It's 106 miles to Chicago, we've got a full tank of gas, half a pack of cigarettes, it's dark and we're wearing sunglasses. Hit it #bluesBrothers",
 '@saraproblemsir Det ligner kridtstre

In [242]:
merged[merged["topic_nr"] == number]["topics_str"].reset_index(drop=True)[0]

"['kold', 'sne', 'varme', 'sidde', 'bil', 'vand', 'vej', 'hund', 'bo', 'hjem']"

### Thesis

In [138]:
df = pd.read_csv("clean_GROUP1_topic_model_testset.csv")

def clean_tweets(row):
    tweet = row["clean_text"]
    clean_tweet = re.sub(r'^rt', '', tweet) #RT
    clean_tweet = re.sub(r'[0-9]+', '', clean_tweet) # numbers
    
    if len(clean_tweet.split()) > 3:
        return clean_tweet
    else:
        return 1
df["cleaner_text"] = df.apply(lambda row: clean_tweets(row), axis = 1)
    
print(len(df))
print(len(df[df["cleaner_text"] != 1]))

df = df[df["cleaner_text"] != 1]
test_documents = df["cleaner_text"].to_list()

39104
38029


In [139]:
testing_dataset = tp.create_test_set(test_documents) # create dataset for the testset

Batches:   0%|          | 0/191 [00:00<?, ?it/s]

In [140]:
# n_sample how many times to sample the distribution (see the documentation)
italian_topics_predictions = ctm.get_thetas(testing_dataset, n_samples=5) # get all the topic predictions

/home/commando/maris/lib/python3.6/site-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:30,  6.18s/it]


In [141]:
topic_number = np.argmax(italian_topics_predictions[0]) # get the topic id of the first document
ctm.get_topic_lists(10)[topic_number]

['gode',
 'god',
 'tak',
 'rigtig',
 'virkelig',
 'faktisk',
 'rette',
 'sikker',
 'ting',
 'ok']

In [142]:
topics = []
for tweet in range(0, len(test_documents)):
    topic_number = np.argmax(italian_topics_predictions[tweet]) # get the topic id of the first document
    topics.append(ctm.get_topic_lists(10)[topic_number])

In [143]:
len(test_documents)

38029

In [144]:
#dff = df[0:40]
df["topics"] = topics

In [ ]:
#df.to_csv("TM_output_Tobias.csv", index = False, header = True)

In [77]:
#from icecream import ic
#for row in range(0, len(dff.iloc[0:10])):
#    ic(dff.iloc[row][2])
#    ic(dff.iloc[row]["topics"])

In [145]:
len(df)

38029

In [146]:
df["topics_str"] = df["topics"].astype(str)
len(df.drop_duplicates("topics_str"))

50

In [147]:
dff = df[["created_at", "id", "text", "from_user_id", "topics_str"]]

In [148]:
topics_df = dff.drop_duplicates("topics_str").reset_index()

In [149]:
topics_df = topics_df.rename(columns={"index":"topic_nr"})

In [150]:
merged = pd.merge(dff, topics_df, how="left", on=["topics_str"])

In [151]:
merged = merged.drop(["created_at_y", "id_y", "text_y", "from_user_id_y"], axis = 1).rename(columns={"created_at_x":"created_at", "id_x":"id","text_x":"text","from_user_id_x":"from_user_id"})

In [152]:
merged = merged.drop_duplicates().reset_index(drop=True)

In [86]:
#merged.to_csv("TM_output_GROUP1.csv", index = False, header = True)

In [87]:
#pd.read_csv("TM_output_GROUP1.csv")

,created_at,id,text,from_user_id,topics_str,topic_nr
0,2020-06-02 05:40:22,1.267693e+18,"RT @sataylite: Meget interessant at se, hvor m...",2.221147e+08,"['hele', 'hver', 'tid', 'fald', 'tak', 'gang',...",0
1,2020-06-02 05:43:36,1.267693e+18,RT @PedeB1984: 9 fucking minutter... Jeg synes...,9.174765e+08,"['sidste', 'ny', 'gang', 'aften', 'uge', 'tid'...",1
2,2020-06-02 05:43:37,1.267693e+18,RT @PukDamsgaard: Talrige dokumenterede angreb...,2.221147e+08,"['stemme', 'politiker', 'politisk', 'politi', ...",2
3,2020-06-02 08:11:33,1.267731e+18,RT @Heeger: Hvis bare vi havde en internationa...,1.000199e+08,"['tro', 'faktisk', 'rigtig', 'sikker', 'vist',...",3
4,2020-06-02 09:05:35,1.267744e+18,RT @TobiasPetersen_: Der gemmer sig megen sand...,2.659420e+07,"['ord', 'bruge', 'feminist', 'feminisme', 'fin...",4
...,...,...,...,...,...,...
38024,2020-02-19 21:45:55+00:00,1.230247e+18,RT @RhHofs: @JakobEllemann Hvis du ser videoen...,1.183859e+18,"['danmark', 'dansk', 'sige', 'dansker', 'vigti...",11
38025,2020-02-19 20:18:21+00:00,1.230225e+18,RT @TheBitcoinFight: @PHummelgaard @JakobEllem...,1.183859e+18,"['briller', 'synd', 'sindssyg', 'vanvittig', '...",42
38026,2020-02-19 20:18:08+00:00,1.230225e+18,RT @TheBitcoinFight: https://t.co/i2rY2UV08S\n...,1.183859e+18,"['kalde', 'hvide', 'racisme', 'sort', 'racisti...",6
38027,2020-02-19 20:17:53+00:00,1.230225e+18,RT @TheBitcoinFight: de (Islamiske Trossamfund...,1.183859e+18,"['spille', 'kamp', 'klub', 'hold', 'vinde', 'f...",81


In [153]:
number = 6
merged[merged["topic_nr"] == number]["text"].to_list()

['RT @BrianMork: Sorte fredelige demonstranter er begyndt at fange antifamedlemmer og udlevere dem til politiet. Siger det ikke lidt om hvem Antifa er?',
 'RT @LylloffMonica: @mortenlorenzen Tænk, hvis sygehusene lod være med at behandle deres patienter, fordi de ikke “havde råd” eller havde fejl i 51% deres behandlinger. Ville man så bare lade stå til? @Astridkrag @handicapdk @kommunerne @regionerne #enmillionstemmer #dkpol',
 'RT @AkhtarMarya: Det er forkert at se ansigtsgenkendelse som noget mere af den overvågning, vi allerede kender. Ansigtet åbner nemlig op fo…',
 'RT @BertelsenMikael: Staten har besluttet at Radio24syv ikke må sagsøge staten.\n\n»Den her sag er for mig at se af en karakter, så selv Kafk…',
 'RT @bennochr: Her er Jeppe. Han var ikke dygtig nok som politiker,så han blev stemt ud. Nu lever han af eftervederlag på 51.000 kr/md og så…',
 'RT @IreneRoger5: Lige på dette punkt og at lægge øre til dette giver mig voldsom kvalme - ALDRIG som i ALDRIG skal der et islamisk

In [154]:
merged[merged["topic_nr"] == number]["topics_str"].reset_index(drop=True)[0]

"['lov', 'stemme', 'folketing', 'politiker', 'regering', 'politisk', 'minister', 'sige', 'grundlov', 'sag']"

## Evaluation

In [278]:
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
from contextualized_topic_models.evaluation.measures import CoherenceCV

texts = preprocessed_documents
texts = [d.split() for d in preprocessed_documents]

npmi = CoherenceCV(texts=texts, topics=ctm.get_topic_lists(10))
npmi.score()

0.42607305348359015

In [55]:
type(preprocessed_documents)

list

In [156]:
texts = testing_dataset
texts = [d.split() for d in testing_dataset]

npmi = CoherenceCV(texts=texts, topics=ctm.get_topic_lists(10))
npmi.score()

AttributeError: 'dict' object has no attribute 'split'

In [160]:
testing_dataset

In [158]:
preprocessed_documents

['vare parre corona',
 'barn',
 'dejlig talent',
 'passe varme',
 'grin ende dag twitter nyde sidste ungdom',
 'mor risikere barn liv desuden bryde lov angribe fri samfund this shit up',
 'dag',
 'fuck tage rar',
 'al anholde kort kvindelig',
 'behandle klage system system',
 'forlade politisk vilje barn barn danmark',
 'folk gemme offentlig profil diverse social medium',
 'tilgang tilgang sige muslimsk dansk hvilken dansk',
 'egentlig litt bekymre',
 'person twitter glad gift betale dag',
 'tusind tak arbejde rigelig igang analysere',
 'ramme big apple mia',
 'lag meste',
 'fortjene samtlige kategori hver fremover gang',
 'tak fjolle liv simpelthen bruge lange tid kigge',
 'huske vaccine tid',
 'omgang kigge handle milliard milliard lange bane sund ny kultur',
 'fyr arbejde engang spilde finte spilde kontor',
 'russisk splitte idiot spille agenda',
 'sjov klog pleje ende gode sted liv',
 'lyd sidde orden mene you know sejle ob',
 'barn land forlade danmark stat pludselig dansk stat he